In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import gc, math
import pickle
import collections

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Model, load_model, model_from_json

from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator
from cnnmodels import identity_block, testcv_generator, conv_block, resnet50_model

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Params
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
ROWS, COLS = 224, 224
CHECKPOINT_DIR = 'log/checkpoint05/'
BATCHSIZE = 32
CERV_CLASSES = ['Type_1', 'Type_2', 'Type_3']
nb_perClass = int(BATCHSIZE / len(CERV_CLASSES))
TRAIN_DIR = '../data/rfcn_crop/train'
TEST_DIR = '../data/rfcn_crop/test'
DATA_DIR = '../data/rfcn_crop'
num_class = len(CERV_CLASSES)
full = False
bags = 5

In [3]:
train_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

In [4]:
img_ls = []
y_ls = []
for typ in CERV_CLASSES:
    for img in os.listdir(os.path.join(TRAIN_DIR, typ)):
        if img != '.DS_Store':
            img_ls.append(os.path.join(TRAIN_DIR, typ, img))
            y_ls.append(typ)
for typ in CERV_CLASSES:
    for img in os.listdir(os.path.join(DATA_DIR, typ)):
        if img != '.DS_Store':
            img_ls.append(os.path.join(DATA_DIR, typ, img))
            y_ls.append(typ)
train_all  = pd.DataFrame({'class': y_ls, 'img': img_ls, })[['img', 'class']]

In [5]:
img_ls = []
for img in os.listdir(TEST_DIR):
    if img != '.DS_Store':
        img_ls.append(os.path.join(TEST_DIR, img))
test_df  = pd.DataFrame({'img': img_ls}) 

In [6]:
train_all.tail(3)

,img,class
6767,../data/rfcn_crop/Type_3/5002.jpg,Type_3
6768,../data/rfcn_crop/Type_3/5518.jpg,Type_3
6769,../data/rfcn_crop/Type_3/5476.jpg,Type_3


In [7]:
def train_generator(datagen, df):
    while 1:
        batch_x = np.zeros((BATCHSIZE, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BATCHSIZE, len(CERV_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nb_perClass, replace=False),:]
        batch_df = df.groupby('class', as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = row[0]
            typ_class = row[1]
            img = Image.open(image_file).resize((ROWS, COLS))
            img = img.convert('RGB')
            x = np.asarray(img, dtype=K.floatx())
            #x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i,CERV_CLASSES.index(typ_class)] = 1
            i += 1
        #return (batch_x, batch_y)
        yield (batch_x.transpose(0, 3, 1, 2), batch_y)

In [8]:
# Split into train and valid
valid_set = pd.read_csv("../val_images.csv", header = None, names = ['img']).img.tolist()
valid_set[-4:]

['Type_2/3498.jpg', 'Type_2/1341.jpg', 'Type_3/6017.jpg', 'Type_2/5629.jpg']

In [9]:
valid_df = train_all[train_all['img'].str.replace('../data/rfcn_crop/', '').isin(valid_set)]
if full == True:
    train_df = train_all
else:
    train_df = train_all[~train_all['img'].str.replace('../data/rfcn_crop/', '').isin(valid_set)]
samples_per_epoch=BATCHSIZE*math.ceil(train_df.groupby('class').size()['Type_2']/nb_perClass)
print(train_df.shape)
print(valid_df.shape)

(5458, 2)
(1312, 2)


In [10]:
# Make our validation set
l = valid_df.groupby('class').size()
valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
valid_y = np.zeros((valid_df.shape[0], len(CERV_CLASSES)), dtype=K.floatx())
i = 0
for index,row in valid_df.iterrows():
    row = row.tolist()
    image_file = row[0]
    typ_class = row[1]
    img = Image.open(image_file).resize((ROWS, COLS))
    img = img.convert('RGB')
    x = np.asarray(img, dtype=K.floatx())
    # x = datagen.random_transform(x)
    x = preprocess_input(x)
    valid_x[i] = x
    valid_y[i,CERV_CLASSES.index(typ_class)] = 1
    i += 1
valid_x = valid_x.transpose(0, 3, 1, 2)

In [11]:
def test_generator(df, datagen, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = row[0]
            # typ_class = row[1]
            img = Image.open(image_file).resize((ROWS, COLS))
            img = img.convert('RGB')
            x = np.asarray(img, dtype=K.floatx())
            # x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            i += 1
        if batch_index%100 == 0: print(batch_index)
        # return (batch_x.transpose(0, 3, 1, 2))
        yield(batch_x.transpose(0, 3, 1, 2))

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')        
model_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR+'weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)
# tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=False, write_images=True)

In [13]:
print "Model creation... "
nb_epoch = 2
model = resnet50_model(ROWS, COLS, channel, num_class)
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-3:]:
    layer.trainable = True

# Start Fine-tuning
print "Fine tune part 1"
model.fit_generator(train_generator(train_datagen, train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch, #50000,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[early_stopping, model_checkpoint, learningrate_schedule],
          )

Model creation... 
Fine tune part 1
Epoch 1/20
9536/9536 [==============================] - 592s - loss: 0.9379 - acc: 0.5252 - val_loss: 1.0591 - val_acc: 0.4459
Epoch 2/20
9536/9536 [==============================] - 593s - loss: 0.7409 - acc: 0.6618 - val_loss: 1.0186 - val_acc: 0.4924
Epoch 3/20
9536/9536 [==============================] - 589s - loss: 0.6182 - acc: 0.7202 - val_loss: 1.0116 - val_acc: 0.5099
Epoch 4/20
9536/9536 [==============================] - 593s - loss: 0.5228 - acc: 0.7735 - val_loss: 1.0083 - val_acc: 0.5221
Epoch 5/20
9536/9536 [==============================] - 575s - loss: 0.4256 - acc: 0.8150 - val_loss: 1.0391 - val_acc: 0.5244
Epoch 6/20
9536/9536 [==============================] - 591s - loss: 0.3465 - acc: 0.8450 - val_loss: 1.0325 - val_acc: 0.5450
Epoch 7/20
9536/9536 [==============================] - 593s - loss: 0.2849 - acc: 0.8724 - val_loss: 1.0115 - val_acc: 0.5709
Epoch 8/20
9504/9536 [============================>.] - ETA: 1s - loss: 0.2

KeyboardInterrupt: 

In [14]:
# Hack to solve issue on model loading : https://github.com/fchollet/keras/issues/4044
import glob
import h5py
model_files = sorted(glob.glob(CHECKPOINT_DIR + '*.hdf5'))
for model_file in model_files:
    print("Update '{}'".format(model_file))
    with h5py.File(model_file, 'a') as f:
        if 'optimizer_weights' in f.keys():
            del f['optimizer_weights']

Update 'log/checkpoint05/weights.000-1.0398.hdf5'
Update 'log/checkpoint05/weights.000-1.0591.hdf5'
Update 'log/checkpoint05/weights.001-1.0032.hdf5'
Update 'log/checkpoint05/weights.001-1.0186.hdf5'
Update 'log/checkpoint05/weights.002-1.0116.hdf5'
Update 'log/checkpoint05/weights.003-1.0083.hdf5'


### Try a prediction from a single epoch

In [15]:
model = load_model(CHECKPOINT_DIR + 'weights.009-0.2856.hdf5')

IOError: Unable to open file (Unable to open file: name = 'log/checkpoint05/weights.009-0.2856.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
test_preds = model.predict_generator(test_generator(test_df, train_datagen), 
                                         val_samples = test_df.shape[0])

In [ ]:
test_sub = pd.DataFrame(test_preds, columns=CERV_CLASSES)
test_sub['image_name'] = test_df['img'].str.split('/').apply(lambda x: x[-1])
test_sub = test_sub[['image_name'] + CERV_CLASSES ]
test_sub.head(3)

In [ ]:
import time
timestr = time.strftime("%Y%m%d")
if full:
    subm_name = '../sub/sub_dara_full_resnet_raw_' + timestr + '.csv' #'.csv.gz'
else:
    subm_name = '../sub/sub_dara_part_resnet_raw_' + timestr + '.csv' #'.csv.gz'

In [ ]:
from IPython.lib.display import FileLink
#test_sub.to_csv(subm_name, index=False)#, compression='gzip')
#FileLink(subm_name)

### Bag the predictions from a few epochs

In [ ]:
import glob
files = glob.glob(CHECKPOINT_DIR+'*')
# there is apparently overfitting on the later epochs so exclude the epochs where we unfroze the top layers
files = [f for f in files if float(f.split('-')[-1][:-5])>0.26]
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
min_id = np.array(val_losses).argsort()[:bags].tolist()

In [ ]:
# Loop the lowest val losses and get a prediction for each
test_preds_ls = []
for index in min_id:
    print('Loading model from checkpoints file ' + files[index])
    test_model = load_model(files[index])
    test_model_name = files[index].split('/')[-2][-1:]+'_'+files[index].split('/')[-1]
    test_preds_ls.append(test_model.predict_generator(test_generator(test_df, train_datagen), 
                                         val_samples = test_df.shape[0])) 
    del test_model
    gc.collect()

In [ ]:
test_preds = sum(test_preds_ls)/len(test_preds_ls)

In [ ]:
test_sub = pd.DataFrame(test_preds, columns=CERV_CLASSES)
test_sub['image_name'] = test_df['img'].str.split('/').apply(lambda x: x[-1])
test_sub = test_sub[['image_name'] + CERV_CLASSES ]
test_sub.head(3)

In [ ]:
if full:
    subm_name = '../sub/sub_dara_full_resnet_raw_5xbag_' + timestr + '.csv' #'.csv.gz'
else:
    subm_name = '../sub/sub_dara_part_resnet_raw_5xbag_' + timestr + '.csv' #'.csv.gz'
    
#test_sub.to_csv(subm_name, index=False)#, compression='gzip')
#FileLink(subm_name)